<a href="https://colab.research.google.com/github/ykitaguchi77/FundusPhoto/blob/main/SeqNet1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Running Github codes (SeqNet) in Google Colab



In [1]:
#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Githubよりリポジトリをcloneする
!git clone https://github.com/conscienceli/SeqNet.git

Cloning into 'SeqNet'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 48 (delta 15), reused 41 (delta 10), pack-reused 0
Unpacking objects: 100% (48/48), done.


#作業ディレクトリを移動

In [4]:
import os
import shutil

model_path = '/content/SeqNet'

#作業ディレクトリをpathに移動する
os.chdir(model_path)

#作業ディレクトリ直下のファイルを確認
!ls

LICENSE  predict.py  requirements-conda.txt  train.py
pics	 README.md   requirements-pip.txt    utils


In [5]:
#評価画像用のフォルダを作成
data_dir = "./data/test_images/"

if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.makedirs(data_dir)

#copy example data
#shutil.copy("/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_img/img00085008_00_1R.jpg", "./data/test_images/image1.jpg")

In [7]:
#入力用フォルダを定義
orig_path = "/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_img"
print("total images: "+str(len(os.listdir(orig_path))))
img_num = list(range(0, 1, 1)) #処理する画像の番号を指定(0~14141)
img_list = []
for i in img_num:
    img_list.append(orig_path+"/"+os.listdir(orig_path)[i])
print(img_list)


#出力用フォルダを作る
dst_path = "/content/drive/MyDrive/Deep_learning/FundusPhoto/vascular_img"
os.makedirs(dst_path, exist_ok=True)

total images: 1414
['/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_img/img14980414_02_1R.jpg']


In [8]:
#Download pretrained model from Gdrive
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

file_id = '1OYjzu0gixtga6e7Rvb2mZoSSYJkXWRNB'

os.makedirs('./trained_model/ALL', exist_ok=True)
destination = './trained_model/ALL/Final_Emer_Iteration_3_cropsize_128_epochs_200.hdf5'
download_file_from_google_drive(file_id, destination)


#upgrade modules
!pip3 install --upgrade tensorflow==1.15.0
!pip3 install --upgrade keras==2.3.1
!pip3 install --upgrade h5py==2.10.0

#stride_sizeを増やす事によるタイムコスト削減
!cat -n predict.py # -n はnumberの意味
!sed -i -e "133c predict(batch_size=24, epochs=200, iteration=3, stride_size=10, crop_size=128," predict.py

     |████████████████████████████████| 412.3 MB 24 kB/s 
     |████████████████████████████████| 503 kB 86.5 MB/s 
     |████████████████████████████████| 3.8 MB 40.1 MB/s 
     |████████████████████████████████| 50 kB 9.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=2dd312c0ab7c082f6d3e0290f446b25f258808bd7f367f962509b45bc8bc7c85
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:
"""
#ここは不要
!cat -n requirements-pip.txt # -n はnumberの意味

!sed -i -e "43c protobuf==3.11.3" requirements-pip.txt # protobufはバージョン違いしかないので変更
!sed -i -e "47d" requirements-pip.txt # pyGPUはサポートされていないので除外

!cat -n requirements-pip.txt # -n はnumberの意味

!pip3 install -r requirements-pip.txt
"""


#**Run codes**

In [ ]:
!python predict.py -i /content/drive/MyDrive/Deep_learning/FundusPhoto/flipped_img/ -o /content/drive/MyDrive/Deep_learning/FundusPhoto/vascular_img/

Using TensorFlow backend.



2021-09-07 15:23:47.877305: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-09-07 15:23:47.881365: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000134999 Hz
2021-09-07 15:23:47.881622: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56538cf5ca00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-07 15:23:47.881656: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-07 15:23:47.883227: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-09-07 15:23:48.067137: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n